In [ ]:
pip install streamlit transformers huggingface_hub peft torch datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 867.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
%%writefile my.py

import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from peft import PeftModel, PeftConfig
from datetime import datetime
import torch

# Set up Hugging Face token directly
HF_TOKEN = ""  # Replace with your Hugging Face token
MODEL_NAME = "iyashnayi/llama-3.2-1B-finetuned"
login(token=HF_TOKEN)

# Configure page layout
st.set_page_config(page_title="SocioLens", layout="wide")

# Custom styling - Updated Theme
st.markdown("""
    <style>
        /* Sidebar styling */
        /* Changed: Set sidebar to light black (#1C2526) for a subtle dark tone */
        section[data-testid="stSidebar"] {
            background-color: #1C2526;
            color: #FFFFFF;
            display: flex;
            flex-direction: column;
        }
        /* Main pane styling */
        /* Changed: Set main pane to pure black (#000000) as requested */
        .main {
            background-color: #000000;
            font-family: 'Inter', sans-serif;
            color: #FFFFFF; /* Ensure main pane text is white for visibility */
            position: relative;
            min-height: 100vh;
        }
        /* Header container */
        .header-container {
            display: flex;
            justify-content: center;
            align-items: center;
            margin-top: 2rem;
            margin-bottom: 1rem;
        }
        /* Title styling */
        /* Changed: Set title to light orange (#FF9F1C) for vibrancy */
        .header-logo {
            font-size: 3rem;
            font-weight: bold;
            color: #FF9F1C;
            font-family: 'Inter', sans-serif;
            letter-spacing: -1px;
            margin-bottom: 0.5rem;
        }
        /* Subtitle styling */
        /* Changed: Set subtitle to light orange (#FF9F1C) to match title */
        .header-subtitle {
            font-size: 1.2rem;
            color: #FF9F1C;
            font-style: italic;
            text-align: center;
            opacity: 0;
            animation: fadeIn 1.5s forwards;
            animation-delay: 0.5s;
        }
        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(10px); }
            to { opacity: 1; transform: translateY(0); }
        }
        /* Button styling */
        /* Changed: Adjusted button background to dark gray (#444444) for contrast on light black sidebar */
        .stButton>button {
            background-color: #444444;
            color: white;
            border: none;
            border-radius: 6px;
            padding: 0.5rem 1rem;
            font-weight: 500;
        }
        .stButton>button:hover {
            background-color: #555555; /* Slightly lighter gray for hover effect */
        }
        /* Chat message styling */
        /* Changed: Set chat message background to dark gray (#2D2D2D) for visibility on pure black main pane */
        .stChatMessage {
            background-color: #2D2D2D;
            border-radius: 10px;
            padding: 15px;
            margin-bottom: 15px;
            border-left: 3px solid #FF9F1C; /* Match light orange theme */
            color: #FFFFFF; /* Ensure text is white */
        }
        .chat-area {
            padding-bottom: 80px;
        }
        .sidebar-content {
            flex: 1;
        }
        .sidebar-upload {
            margin-top: auto;
            padding-bottom: 20px;
            /* Changed: Adjusted border color to light gray (#444444) for visibility */
            border-top: 1px solid #444444;
            padding-top: 15px;
        }
        /* File uploader styling */
        /* Changed: Set uploader background to darker shade (#2D2D2D) for contrast */
        .stSidebar div[data-testid="stFileUploader"] {
            background-color: #2D2D2D;
            border-radius: 8px;
            padding: 10px;
            margin-top: 10px;
        }
        .stSidebar div[data-testid="stFileUploader"] label {
            color: #FFFFFF; /* Ensure label is white */
            font-weight: bold;
        }
        /* Upload button styling */
        /* Changed: Adjusted upload button color to match theme (#444444) */
        .upload-button {
            width: 100%;
            background-color: #444444;
            color: white;
            border: none;
            border-radius: 6px;
            padding: 8px;
            font-weight: 500;
            margin-top: 5px;
            cursor: pointer;
            text-align: center;
        }
        /* Sidebar header styling */
        /* Changed: Set sidebar headers to white (#FFFFFF) for visibility on light black */
        .sidebar-header {
            color: #FFFFFF;
            font-weight: 600;
            margin-top: 1rem;
        }
        /* Hide unnecessary elements */
        .element-container:has(footer) {display: none;}
        #MainMenu {visibility: hidden;}
        header {visibility: hidden;}
    </style>
""", unsafe_allow_html=True)

# Header
st.markdown("""
    <div class="header-container">
        <div>
            <div class="header-logo">💬 SocioLens</div>
            <div class="header-subtitle">How may I help you today?</div>
        </div>
    </div>
""", unsafe_allow_html=True)

st.markdown("<style>h1 {display: none;}</style>", unsafe_allow_html=True)

@st.cache_resource
def load_model():
    peft_model_id = MODEL_NAME
    config = PeftConfig.from_pretrained(peft_model_id)

    base_model = AutoModelForCausalLM.from_pretrained(
        config.base_model_name_or_path,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto"
    )

    model = PeftModel.from_pretrained(base_model, peft_model_id)
    tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
    return model, tokenizer

model, tokenizer = load_model()

# Session state setup
if "messages" not in st.session_state:
    st.session_state.messages = []
if "chat_history" not in st.session_state:
    st.session_state.chat_history = {}
if "uploaded_file" not in st.session_state:
    st.session_state.uploaded_file = None

# Sidebar
with st.sidebar:
    st.markdown('<div class="sidebar-content">', unsafe_allow_html=True)
    st.markdown('<h3 class="sidebar-header">📁 Chat Options</h3>', unsafe_allow_html=True)
    if st.button("🆕 New Chat"):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        if st.session_state.messages:
            st.session_state.chat_history[timestamp] = st.session_state.messages
        st.session_state.messages = []
        st.rerun()

    st.markdown('<h3 class="sidebar-header">📜 Chat History</h3>', unsafe_allow_html=True)
    for k in reversed(list(st.session_state.chat_history.keys())):
        if st.button(k):
            st.session_state.messages = st.session_state.chat_history[k]
            st.rerun()
    st.markdown('</div>', unsafe_allow_html=True)

    st.markdown('<div class="sidebar-upload">', unsafe_allow_html=True)
    st.markdown('<h3 class="sidebar-header">📎 Upload Files</h3>', unsafe_allow_html=True)
    uploaded_file = st.file_uploader("Upload document to analyze", type=["txt", "pdf", "docx"])
    if uploaded_file:
        st.session_state.uploaded_file = uploaded_file
        # Changed: Set success message text to white for visibility
        st.markdown(f'<p style="color: white;">Uploaded: {uploaded_file.name}</p>', unsafe_allow_html=True)
        file_details = {
            "Filename": uploaded_file.name,
            "File size": f"{uploaded_file.size / 1024:.2f} KB"
        }
        # Changed: Set file details text to white for visibility
        st.markdown('<p style="color: white;">File Details:</p>', unsafe_allow_html=True)
        for key, value in file_details.items():
            st.markdown(f'<p style="color: white;">- <b>{key}:</b> {value}</p>', unsafe_allow_html=True)
    st.markdown('</div>', unsafe_allow_html=True)

# Chat area
chat_area = st.container()
with chat_area:
    st.markdown('<div class="chat-area">', unsafe_allow_html=True)
    for msg in st.session_state.messages:
        with st.chat_message(msg["role"]):
            st.markdown(f'<p style="color: white;">{msg["content"]}</p>', unsafe_allow_html=True)  # Changed: Ensure chat text is white
    st.markdown('</div>', unsafe_allow_html=True)

# Chat input
if prompt := st.chat_input("Type your message..."):
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.rerun()

# Process assistant response
if st.session_state.messages and st.session_state.messages[-1]["role"] == "user":
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            prompt = st.session_state.messages[-1]["content"]
            inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=200)
            reply = tokenizer.decode(outputs[0], skip_special_tokens=True)
            st.markdown(f'<p style="color: white;">{reply}</p>', unsafe_allow_html=True)  # Changed: Ensure reply text is white
            st.session_state.messages.append({"role": "assistant", "content": reply})

Writing my.py


In [ ]:
# Install necessary packages
!pip install streamlit
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

# Kill all existing tunnels
ngrok.kill()


In [ ]:
# Authenticate ngrok with your token
!ngrok authtoken ""  # Replace this with your actual ngrok token



# Run Streamlit app in the background and expose it via ngrok
from pyngrok import ngrok
import os

# Run Streamlit app (background execution)
os.system("streamlit run my.py &")

# Open ngrok tunnel for the Streamlit app (exposing port 8501)
public_url = ngrok.connect(8501)
print(f"Streamlit app is running at: {public_url}")
